In [ ]:
import warnings
import cv2
import torch
import torchvision.models.video as models
import os
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display, Video, clear_output
import warnings
warnings.filterwarnings('ignore')

# Suppress warnings
warnings.filterwarnings("ignore")

# Load the pretrained 3D CNN model
cnn_model = models.r3d_18(pretrained=True)

# Set the model to evaluation mode
cnn_model.eval()

# Preprocess video frames
def preprocess_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert frame to RGB
        frame = cv2.resize(frame, (224, 224))  # Resize frame to (224, 224) for compatibility with the model
        frames.append(frame)
    cap.release()
    video_frames = torch.tensor(frames, dtype=torch.float).permute(3, 0, 1, 2) / 255.0  # Normalize pixel values to [0, 1]
    video_frames = video_frames.unsqueeze(0)  # Add batch dimension
    return video_frames

# Function to generate video embedding
def generate_video_embedding(video_path):
    video_frames = preprocess_video(video_path)
    with torch.no_grad():
        output = cnn_model(video_frames)
    return output.squeeze().numpy()

# Function for semantic search
def semantic_search(query_embedding, video_folder, top_n=2):
    similarities = []
    for filename in os.listdir(video_folder):
        if filename.endswith('.mp4'):
            video_path = os.path.join(video_folder, filename)
            # Generate embedding for the video
            video_embedding = generate_video_embedding(video_path)
            # Compute cosine similarity between query embedding and video embedding
            similarity = cosine_similarity([query_embedding], [video_embedding])[0][0]
            similarities.append((filename, similarity))
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:top_n]

# Test
query_video_folder = "/usr/local/datasetsDir/video-dataset/queries"  # Path to the query video
video_folder = "/usr/local/datasetsDir/video-dataset/data"  # Path to the folder containing videos
for filename in os.listdir(query_video_folder):
  if filename.endswith('.mp4'):
    print("Query Video: ", filename)
    query_video_path = os.path.join(query_video_folder, filename)
    query_embedding = generate_video_embedding(query_video_path)
    display(Video(query_video_path, embed=True))
    results = semantic_search(query_embedding, video_folder)
    print("Top matching videos: ")
    for filename, similarity in results:
      print(f"{filename}, Similarity: {similarity:.2f}")
      display(Video(os.path.join(video_folder, filename), embed=True))